In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler()\
  .setInputCols(["Quantity", "UnitPrice"])\
  .setOutputCol("features")

sales = va.transform(spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("file:///home/ubuntu/Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")
  .limit(50)
  .coalesce(1)
  .where("Description IS NOT NULL"))

sales.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, features: vector]

In [3]:
from pyspark.ml.clustering import KMeans
km = KMeans().setK(5)
print(km.explainParams())
kmModel = km.fit(sales)

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2, current: 5)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 7969353092125344463)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)


In [4]:
summary = kmModel.summary
print(summary.clusterSizes) # number of points
kmModel.computeCost(sales)
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

[10, 8, 29, 2, 1]
Cluster Centers: 
[23.2    0.956]
[ 2.5     11.24375]
[7.55172414 2.77172414]
[48.    1.32]
[36.    0.85]


In [5]:
from pyspark.ml.clustering import BisectingKMeans
bkm = BisectingKMeans().setK(5).setMaxIter(5)
bkmModel = bkm.fit(sales)

In [6]:
summary = bkmModel.summary
print(summary.clusterSizes) # number of points
kmModel.computeCost(sales)
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

[16, 8, 13, 10, 3]
Cluster Centers: 
[23.2    0.956]
[ 2.5     11.24375]
[7.55172414 2.77172414]
[48.    1.32]
[36.    0.85]


In [7]:
from pyspark.ml.clustering import GaussianMixture
gmm = GaussianMixture().setK(5)
print(gmm.explainParams())
model = gmm.fit(sales)

featuresCol: features column name. (default: features)
k: Number of independent Gaussians in the mixture model. Must be > 1. (default: 2, current: 5)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
seed: random seed. (default: -7090211980209472397)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.01)


In [8]:
summary = model.summary
print(model.weights)
model.gaussiansDF.show()
summary.cluster.show()
summary.clusterSizes
summary.probability.show()

[0.16503937777770641, 0.35496420094056985, 0.06003637101912308, 0.1999636297743671, 0.21999642048823354]
+--------------------+--------------------+
|                mean|                 cov|
+--------------------+--------------------+
|[2.54180583818530...|0.785769315153778...|
|[5.07243095740621...|2.059950971034034...|
|[43.9877864408847...|32.22707068867282...|
|[23.1998836372414...|2.560279258630084...|
|[11.6364190345020...|1.322132750446848...|
+--------------------+--------------------+

+----------+
|prediction|
+----------+
|         2|
|         3|
|         3|
|         3|
|         1|
|         2|
|         4|
|         3|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         4|
|         1|
|         2|
|         3|
|         1|
|         1|
+----------+
only showing top 20 rows

+--------------------+
|         probability|
+--------------------+
|[1.37632400885157...|
|[4.89041912245635...|
|[1.67299627008735...|
|[7.43321003719004...|

In [9]:
from pyspark.ml.feature import Tokenizer, CountVectorizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.drop("features"))
cv = CountVectorizer()\
  .setInputCol("DescOut")\
  .setOutputCol("features")\
  .setVocabSize(500)\
  .setMinTF(0)\
  .setMinDF(0)\
  .setBinary(True)
cvFitted = cv.fit(tokenized)
prepped = cvFitted.transform(tokenized)

In [10]:
from pyspark.ml.clustering import LDA
lda = LDA().setK(10).setMaxIter(5)
print(lda.explainParams())
model = lda.fit(prepped)

checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
docConcentration: Concentration parameter (commonly named "alpha") for the prior placed on documents' distributions over topics ("theta"). (undefined)
featuresCol: features column name. (default: features)
k: The number of topics (clusters) to infer. Must be > 1. (default: 10, current: 10)
keepLastCheckpoint: (For EM optimizer) If using checkpointing, this indicates whether to keep the last checkpoint. If false, then the checkpoint will be deleted. Deleting the checkpoint can cause failures if a data partition is lost, so set this bit with care. (default: True)
learningDecay: Learning rate, set as anexponential decay rate. This should be between (0.5, 1.0] to guarantee asymptotic convergence. (default: 0.51)
learningOffset: A (pos

In [11]:
model.describeTopics(3).show()
cvFitted.vocabulary

+-----+-------------+--------------------+
|topic|  termIndices|         termWeights|
+-----+-------------+--------------------+
|    0|   [4, 6, 17]|[0.01166617595444...|
|    1|  [13, 9, 66]|[0.01091339619437...|
|    2|[15, 131, 45]|[0.00897001978399...|
|    3| [6, 125, 78]|[0.00902243209856...|
|    4|[103, 55, 62]|[0.00933169978592...|
|    5|  [11, 5, 23]|[0.01496249652116...|
|    6|   [7, 16, 2]|[0.01140631424514...|
|    7| [28, 73, 69]|[0.01535561351823...|
|    8|    [0, 3, 1]|[0.01802752067282...|
|    9| [46, 90, 30]|[0.01169076631125...|
+-----+-------------+--------------------+



['water',
 'hot',
 'vintage',
 'bottle',
 'paperweight',
 '6',
 'home',
 'doormat',
 'landmark',
 'bicycle',
 'frame',
 'ribbons',
 '',
 'classic',
 'rose',
 'kit',
 'leaf',
 'sweet',
 'bag',
 'airline',
 'doorstop',
 'light',
 'in',
 'christmas',
 'heart',
 'calm',
 'set',
 'keep',
 'balloons',
 'night',
 'lights',
 '12',
 'tin',
 'english',
 'caravan',
 'stuff',
 'tidy',
 'oxford',
 'full',
 'cottage',
 'notting',
 'drawer',
 'mushrooms',
 'chrome',
 'champion',
 'amelie',
 'mini',
 'the',
 'giant',
 'design',
 'elegant',
 'tins',
 'jet',
 'fairy',
 "50's",
 'holder',
 'message',
 'blue',
 'storage',
 'tier',
 'covent',
 'world',
 'skulls',
 'font',
 'hearts',
 'skull',
 'clips',
 'bell',
 'red',
 'party',
 'chalkboard',
 'save',
 '4',
 'coloured',
 'poppies',
 'garden',
 'nine',
 'girl',
 'shimmering',
 'doughnut',
 'dog',
 '3',
 'tattoos',
 'chilli',
 'coat',
 'torch',
 'sunflower',
 'tale',
 'cards',
 'puncture',
 'woodland',
 'bomb',
 'knack',
 'lip',
 'collage',
 'rabbit',
 'sex